# Importing libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# Loading data

In [2]:
# Import files
path=r'C:\Users\asicz\OneDrive\Dokumenty\CareerFoundry_Data_Analyst_Course\Data Immersion\Achievement 4\20240508_Instacart_Basket_Analysis'
df_prod = pd.read_csv(os.path.join(path, '02_Data', 'Prepared_data', 'products_cleaned.csv'), index_col = False)
df_ord_prod = pd.read_pickle(os.path.join(path, '02_Data','Prepared_Data', 'orders_products_combined.pkl'))

# Check the data

In [3]:
# Check the df_ord_prod dimensions
df_ord_prod.shape

(32434489, 13)

In [4]:
# Check for the key to match between both dataframes
df_ord_prod.head()

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,1st_order,product_id,add_to_cart_order,reordered,_merge
0,0,0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,0,0,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,0,0,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,0,0,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,0,0,2539329,1,1,2,8,NaN,True,26405,5,0,both


In [5]:
# Check the df_prod dimensions
df_prod.shape

(49672, 6)

In [6]:
# Check for the key to match between both dataframes
df_prod.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3


# Prepare the data to combine
- Looking at the possible keys to match, we see that 'procuct_id' is the only matching column name between the two dataframes.
- Checking the basic stats of the df_prod dataframe below we can see that this data set should contain all the products. Even though the highest product ID is a higher number than the total amount of entries to this table (49688 vs 49672) - therefore some IDs are missing.
- To combine df_ord_prod with df_prod we want to match product IDs in both tables and add additional information about the products to the new ordering table. To achieve this we need to match all the products ID that already exist in the df_ord_prod and therefore we can use left join option.
- I tested also outer join method and I got discrepancy of 11 rows: outer join 32435070 , left join 32435059 --> this means that there are 11 orders that could not match product IDs from the product table. Since we are going to analyze current state of orders we don't need to worry about the products that were never ordered.

In [7]:
# Check the basic stats of df_prod
df_prod.describe()

,Unnamed: 0,product_id,aisle_id,department_id,prices
count,49672.000000,49672.000000,49672.000000,49672.000000,49672.000000
mean,24852.005053,24850.349775,67.762442,11.728942,9.993282
std,14342.265579,14340.705287,38.315784,5.850779,453.615536
min,0.000000,1.000000,1.000000,1.000000,1.000000
25%,12432.750000,12432.750000,35.000000,7.000000,4.100000
50%,24851.500000,24850.500000,69.000000,13.000000,7.100000
75%,37272.250000,37268.250000,100.000000,17.000000,11.100000
max,49692.000000,49688.000000,134.000000,21.000000,99999.000000


In [8]:
# Remove the flag column from the df_ord_prod to be able to flag the newly merged df (name conflict and no use for the flag of the old merge)
df_ord_prod = df_ord_prod.drop(columns=['_merge'])
df_ord_prod

,Unnamed: 0.1,Unnamed: 0,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,1st_order,product_id,add_to_cart_order,reordered
0,0,0,2539329,1,1,2,8,NaN,True,196,1,0
1,0,0,2539329,1,1,2,8,NaN,True,14084,2,0
2,0,0,2539329,1,1,2,8,NaN,True,12427,3,0
3,0,0,2539329,1,1,2,8,NaN,True,26088,4,0
4,0,0,2539329,1,1,2,8,NaN,True,26405,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
32434484,3421081,3421081,2977660,206209,13,1,12,7.0,False,14197,5,1
32434485,3421081,3421081,2977660,206209,13,1,12,7.0,False,38730,6,0
32434486,3421081,3421081,2977660,206209,13,1,12,7.0,False,31477,7,0
32434487,3421081,3421081,2977660,206209,13,1,12,7.0,False,6567,8,0


# Merge the data

In [29]:
# Merge the df_ord_prod and df_prod dataframes using outer join
df_ord_prod_merged_outer = df_ord_prod.merge(df_prod, on = 'product_id', how = 'outer', indicator = True)

In [30]:
df_ord_prod_merged_outer

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,1st_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,_merge
0,1987.0,1987.0,3139998.0,138.0,28.0,6.0,11.0,3.0,False,1,5.0,0.0,0.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,both
1,1989.0,1989.0,1977647.0,138.0,30.0,6.0,17.0,20.0,False,1,1.0,1.0,0.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,both
2,11433.0,11433.0,389851.0,709.0,2.0,0.0,21.0,6.0,False,1,20.0,0.0,0.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,both
3,12198.0,12198.0,652770.0,764.0,1.0,3.0,13.0,NaN,True,1,10.0,0.0,0.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,both
4,12200.0,12200.0,1813452.0,764.0,3.0,4.0,17.0,9.0,False,1,11.0,1.0,0.0,Chocolate Sandwich Cookies,61.0,19.0,5.8,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32435065,3321928.0,3321928.0,1788356.0,200215.0,2.0,0.0,9.0,5.0,False,49688,27.0,0.0,49692.0,Fresh Foaming Cleanser,73.0,11.0,13.5,both
32435066,3324686.0,3324686.0,3401313.0,200377.0,1.0,4.0,11.0,NaN,True,49688,5.0,0.0,49692.0,Fresh Foaming Cleanser,73.0,11.0,13.5,both
32435067,3332310.0,3332310.0,809510.0,200873.0,5.0,3.0,8.0,15.0,False,49688,12.0,0.0,49692.0,Fresh Foaming Cleanser,73.0,11.0,13.5,both
32435068,3332314.0,3332314.0,2359893.0,200873.0,9.0,3.0,15.0,5.0,False,49688,11.0,1.0,49692.0,Fresh Foaming Cleanser,73.0,11.0,13.5,both


In [33]:
# Check for the full match between both dataframes
df_ord_prod_merged_outer['_merge'].value_counts()

_merge
both          32404859
left_only        30200
right_only          11
Name: count, dtype: int64

In [9]:
# Merge the df_ord_prod and df_prod dataframes using left join
df_ord_prod_merged = df_ord_prod.merge(df_prod, on = 'product_id', how = 'left', indicator = True)

In [10]:
df_ord_prod_merged

,Unnamed: 0.1,Unnamed: 0_x,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,1st_order,product_id,add_to_cart_order,reordered,Unnamed: 0_y,product_name,aisle_id,department_id,prices,_merge
0,0,0,2539329,1,1,2,8,NaN,True,196,1,0,195.0,Soda,77.0,7.0,9.0,both
1,0,0,2539329,1,1,2,8,NaN,True,14084,2,0,14084.0,Organic Unsweetened Vanilla Almond Milk,91.0,16.0,12.5,both
2,0,0,2539329,1,1,2,8,NaN,True,12427,3,0,12427.0,Original Beef Jerky,23.0,19.0,4.4,both
3,0,0,2539329,1,1,2,8,NaN,True,26088,4,0,26089.0,Aged White Cheddar Popcorn,23.0,19.0,4.7,both
4,0,0,2539329,1,1,2,8,NaN,True,26405,5,0,26406.0,XL Pick-A-Size Paper Towel Rolls,54.0,17.0,1.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32435054,3421081,3421081,2977660,206209,13,1,12,7.0,False,14197,5,1,14197.0,Tomato Paste,9.0,9.0,5.6,both
32435055,3421081,3421081,2977660,206209,13,1,12,7.0,False,38730,6,0,38734.0,Brownie Crunch High Protein Bar,3.0,19.0,5.9,both
32435056,3421081,3421081,2977660,206209,13,1,12,7.0,False,31477,7,0,31479.0,High Protein Bar Chunky Peanut Butter,3.0,19.0,4.2,both
32435057,3421081,3421081,2977660,206209,13,1,12,7.0,False,6567,8,0,6567.0,Chocolate Peanut Butter Protein Bar,3.0,19.0,4.9,both


In [11]:
# Check for the full match between both dataframes
df_ord_prod_merged['_merge'].value_counts()

_merge
both          32404859
left_only        30200
right_only           0
Name: count, dtype: int64

# Exporting files

In [12]:
# Export data to pkl - regarding the large format and therefore size of the resulting dataframe we use pickle format to save the output
df_ord_prod_merged.to_pickle(os.path.join(path, '02_Data','Prepared_Data', 'ords_prods_merge.pkl'))